# DevMuT Artifact

Artifact of the paper "DevMuT: Testing Deep Learning Framework via Developer
Expertise-Based Mutation" from ASE 2024.

To access ChameleonCloud resources, you may need the account to log in to ChameleonCloud. You also need to have a project to allocate resources (e.g., node).

### Lease creation

We'll create a short lease for one Nvidia Jetson Nano device. Replace project_id with your chameleon cloud project/allocation

In [12]:
import chi
import os

chi.use_site("CHI@Edge")
project_id = 'CHI-251412'
chi.set("project_name",  project_id)

# get your username, just used to name leases something identifiable
username = os.environ.get("USER")

machine_name = "jetson-nano"

from chi import lease

# get dates for lease start and end
start, end = lease.lease_duration(days=1)

# make a unique name for the lease
lease_name = f"{username}-{machine_name}-{start}"

reservations = []
lease.add_device_reservation(reservations, count=1, machine_name=machine_name)
container_lease = lease.create_lease(lease_name, reservations)
lease_id = container_lease["id"]

print(f"created lease with name {lease_name} and uuid {lease_id}, waiting for it to start. This can take up to 60s.")
lease.wait_for_active(lease_id)
print("Done!")

Now using CHI@Edge:
URL: https://chi.edge.chameleoncloud.org
Location: University of Chicago, Chicago, Illinois, USA
Support contact: help@chameleoncloud.org
created lease with name radhofanazizi_gmail_com-jetson-nano-2025-03-24 15:06 and uuid 9de51975-1a20-465a-a71a-e556dbd0a0bc, waiting for it to start. This can take up to 60s.
Done!


### Container creation

Next, we are going to launch a container on our device, ensuring we set the additional `runtime` argument to `nvidia` to make sure that all nvidia compute modules are loaded and the nvidia cuda libraries are mounted onto the container. 

We will be using an nvidia provided PyTorch for Linux4Tegra image (`nvcr.io/nvidia/l4t-pytorch:r32.7.1-pth1.10-py3`) compatible with the current version of L4T installed on our nvidia jetson devices. PyTorch in itself requires a variety of CUDA libraries that are all mounted onto the container when `runtime="nvidia"` is specified during `container.create_container()` call.

***Note***: The container may fail to schedule sometimes due to intermittent failures with devices, this cell is idempotent and can be rerun if needed.

In [13]:
from chi import container

try:
    container.destroy_container(f"tutorial-{machine_name}-nvidia-runtime".replace("_","-"))
except:
    pass

print("Requesting container ... This may take a while as the container image (800MB) is being downloaded")

# set a name for the container. Because CHI@Edge uses Kubernetes, ensure that underscores aren't in the name
container_name = f"tutorial-{machine_name}-nvidia-runtime".replace("_","-")

try:
    my_container = container.create_container(
        container_name, 
        image="nvcr.io/nvidia/l4t-pytorch:r32.7.1-pth1.10-py3",
        command=["/bin/bash", "-c", "while true; do sleep infinity; done"],
        workdir="/home",
        runtime="nvidia",
        reservation_id=lease.get_device_reservation(lease_id),
        platform_version=2,
    )
except RuntimeError as ex:
    print(ex)
    print(f"please stop and/or delete {container_name} and try again")
else:
    print(f"Successfully created container: {container_name}!")

Requesting container ... This may take a while as the container image (800MB) is being downloaded
Waiting for container b07db788-6b09-4ed2-9e6a-a80026f74ff1 status to turn to Running. This can take a while depending on the image
Successfully created container: tutorial-jetson-nano-nvidia-runtime!


## Run DevMuT

Now, we can finally run DevMuT. First we need to clone the github repo first and then run the reprduce.sh script which contain the instructions from README.md
packaged into a bash file

In [14]:
print(container.execute(my_container.uuid, "git clone https://github.com/radhofan/DevMuT.git")["output"])

Cloning into 'DevMuT'...



In [24]:
print(container.execute(my_container.uuid, "apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 16FAAD7AF99A65E2"))
print(container.execute(my_container.uuid, "bash -c 'apt-get update && apt-get install -y curl'")["output"])

{'output': 'Warning: apt-key output should not be parsed (stdout is not a terminal)\nExecuting: /tmp/apt-key-gpghome.UG1X2EvDkO/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 16FAAD7AF99A65E2\ngpg: key A65337CCA8A748B8: "Kitware Apt Archive Automatic Signing Key (2025) <debian@kitware.com>" not changed\ngpg: Total number processed: 1\ngpg:              unchanged: 1\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}
Hit:1 https://apt.kitware.com/ubuntu bionic InRelease
Hit:2 http://ports.ubuntu.com/ubuntu-ports bionic InRelease
Hit:3 http://ports.ubuntu.com/ubuntu-ports bionic-updates InRelease
Hit:4 http://ports.ubuntu.com/ubuntu-ports bionic-backports InRelease
Hit:5 http://ports.ubuntu.com/ubuntu-ports bionic-security InRelease
Reading package lists...


In [68]:
print(container.execute(my_container.uuid, "bash -c 'chmod +x DevMuT/reproduce.sh'")["output"])
print(container.execute(my_container.uuid, "bash -c 'DevMuT/reproduce.sh > reproduce_output.log 2>&1'")["output"])
print(container.execute(my_container.uuid, "bash -c 'cat reproduce_output.log'")["output"])



ERROR: File or directory already exists: '/root/miniconda'
If you want to update an existing installation, use the -u option.
DevMuT/reproduce.sh: line 12: conda: command not found
DevMuT/reproduce.sh: line 13: mamba: command not found
DevMuT/reproduce.sh: line 15: mamba: command not found
DevMuT/reproduce.sh: line 18: mamba: command not found
DevMuT/reproduce.sh: line 19: mamba: command not found
DevMuT/reproduce.sh: line 20: pip: command not found
DevMuT/reproduce.sh: line 21: pip: command not found



In [65]:
print(container.execute(my_container.uuid, "python3 DevMuT/code/DevMuT/mutation_test.py")["output"])

Traceback (most recent call last):
  File "DevMuT/code/DevMuT/mutation_test.py", line 3, in <module>
    import yaml
ModuleNotFoundError: No module named 'yaml'



In [66]:
print(container.execute(my_container.uuid, "ls -l DevMuT/code/DevMuT/common")["output"])

total 92
-rw-r--r-- 1 root root     0 Mar 24 15:08 __init__.py
drwxr-xr-x 2 root root  4096 Mar 24 15:08 __pycache__
-rw-r--r-- 1 root root  8694 Mar 24 15:08 analyzelog_util.py
-rw-r--r-- 1 root root 17952 Mar 24 15:08 dataset_utils.py
-rw-r--r-- 1 root root 11108 Mar 24 15:08 help_utils.py
-rw-r--r-- 1 root root  1462 Mar 24 15:08 log_recoder.py
-rw-r--r-- 1 root root  5742 Mar 24 15:08 loss_utils.py
-rw-r--r-- 1 root root   585 Mar 24 15:08 model_train.py
-rw-r--r-- 1 root root 14016 Mar 24 15:08 model_utils.py
drwxr-xr-x 3 root root  4096 Mar 24 15:08 mutation_ms
drwxr-xr-x 3 root root  4096 Mar 24 15:08 mutation_torch
-rw-r--r-- 1 root root   383 Mar 24 15:08 opt_utils.py



In [69]:
container.destroy_container(my_container.uuid)
lease.delete_lease(lease_id)

AttributeError: 'dict' object has no attribute 'delete'